In [ ]:
import pandas as pd, ipywidgets as widgets, numpy as np, matplotlib.pyplot as plt, io
from adjustText import adjust_text
positive_uploader, negative_uploader, eln_uploader = widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False), widgets.FileUpload(multiple = False)
display("Upload Transfected iQue3 Data (csv)")
display(positive_uploader)
display("Upload Parental iQue3 Data (csv)")
display(negative_uploader)
display("Upload ELN w/ Antibodies Sheet (xlsx)")
display(eln_uploader)

In [ ]:
positivename, negativename, elnname = list(positive_uploader.value.keys())[0], list(negative_uploader.value.keys())[0], list(eln_uploader.value.keys())[0]
positive_iquedata = pd.read_csv(io.BytesIO(positive_uploader.value[iquename]['content']))
negative_iquedata = pd.read_csv(io.BytesIO(negative_uploader.value[iquename]['content']))
eln = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']))
ab_sheet = pd.read_excel(io.BytesIO(eln_uploader.value[elnname]['content']), sheet_name = "Antibody Plate Map")
#positive_iquedata

In [ ]:
positive_data_cleaned = positive_iquedata[["Plate", "Well ID", "Well Type", "Median RL1-H of singlets"]]
#positive_data_cleaned
negative_data_cleaned = negative_iquedata[["Plate", "Well ID", "Well Type", "Median RL1-H of singlets"]]

In [ ]:
#Create a series with only the Positive population from plate, and another series with only those MFIs
#Separate n=1 and n=2 duplicates, average them and reset the indeces 
positives_MFI = positive_data_cleaned["Median RL1-H of singlets"]
firsts = positives_MFI.iloc[::2]
seconds = positives_MFI.iloc[1::2]
firsts.reset_index(inplace = True, drop = True)
seconds.reset_index(inplace = True, drop = True)
averagePositive = firsts + seconds
averagePositive = averagePositive / 2

In [ ]:
#Do the same with the Negative population (untransfected controls)
negatives_MFI = negative_data_cleaned["Median RL1-H of singlets"]
negatives_MFI.reset_index(inplace = True, drop = True)
firsts = negatives_MFI.iloc[::2]
seconds = negatives_MFI.iloc[1::2]
firsts.reset_index(inplace = True, drop = True)
seconds.reset_index(inplace = True, drop = True)
averageNegative = firsts + seconds
averageNegative = averageNegative / 2

#averageNegative

In [ ]:
#Create new DF for MFI ratios(don't modify originals)
MFIratios = averagePositive / averageNegative
positives_MFIratio = positive_data_cleaned.copy()
positives_MFIratio = positives_MFIratio[::2]
positives_MFIratio.reset_index(inplace = True, drop = True)
positives_MFIratio["MFI Ratio"] = MFIratios

MFIRatio_Antibody = pd.concat([positives_MFIratio, ab_sheet], axis = 1)
#MFIRatio_Antibody
MFIRatio_Antibody_cleaned = MFIRatio_Antibody[["Plate", "Well ID", "Well Type", "MFI Ratio", "DNA", "Protein Concentration (ug/uL)"]]
#MFIRatio_Antibody_cleaned = MFIRatio_Antibody_cleaned[MFIRatio_Antibody_cleaned["DNA"].notna()]
MFIRatio_Antibody_cleaned.to_excel("Well_ID_MFI_Ratio.xlsx")
AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
AllLeadsSorted

In [ ]:
#Sort DF by highest MFI ratio (top hits)
AllLeadsSorted = MFIRatio_Antibody_cleaned.sort_values(by="MFI Ratio", ascending = False)
Top10 = MFIRatio_Antibody_cleaned.nlargest(10, "MFI Ratio")
#Top10 = Top10.append(MFIRatio_Antibody_cleaned.loc[MFIRatio_Antibody_cleaned["DNA"] == "SC-175-017"])
fig, ax = plt.subplots()
ax.scatter(x = AllLeadsSorted["Protein Concentration (ug/uL)"], y=AllLeadsSorted["MFI Ratio"], alpha = .2, color = "blue")
ax.scatter(x = Top10["Protein Concentration (ug/uL)"], y = Top10["MFI Ratio"], alpha = .8, color = "blue", label = "Top Hits")
ax.set_title("CD40 Primary Screen")
ax.set_xlabel("Concentration (mg/ml)")
ax.set_ylabel("MFI Ratio \n Transfected/Untransfected")
ax.legend()

# texts = []
# for row in Top10.itertuples(index = False):
#     ab, conc, MFI = row[4], row[5], row[3]
#     ax.annotate(row[4], xy = (conc, MFI), textcoords = "offset points", xytext = (8, -2), ha ='left')
#     #texts.append(plt.text(ab, conc, MFI))